<h1>Web Scrapping</h1>

<h4>L'importation des bibliothèques</h4>
<ul>
<li><p>Le module <strong>requests</strong> vous permet d'envoyer des requêtes HTTP en utilisant Python. La requête HTTP renvoie un objet de réponse avec toutes les données de réponse (contenu, encodage, statut, etc.).</p></li>

<li><p><strong> Beautiful Soup </strong> est une bibliothèque Python utilisée à des fins de grattage Web pour extraire les données des fichiers HTML et XML. Il crée une arborescence d'analyse à partir du code source de la page qui peut être utilisée pour extraire des données de manière hiérarchique et plus lisible.</p></li>


<li><p><strong>Python RegEx.</strong> Une RegEx, ou expression régulière, est une séquence de caractères qui forme un modèle de recherche. RegEx peut être utilisé pour vérifier si une chaîne contient le modèle de recherche spécifié.</p></li>


<li><p>Le format dit <strong>CSV</strong> (Comma Separated Values) est le format d'importation et d'exportation le plus courant pour les feuilles de calcul et les bases de données. ... Le module csv implémente des classes pour lire et écrire des données tabulaires au format CSV.</p></li>
</ul>

In [1]:
import requests
from  bs4 import BeautifulSoup as soup
import csv
import re

<ul>
<li>Dans cette section, on définit la liste <strong>"header"</strong> contenant les 2 "headers", puis un ouvre un fichier CSV qu'on nomme <strong>"hotel_reviews.csv"</strong> en mode écriture dans une variable <strong>data</strong>. Ensuite, on instancie un <strong>writer</strong> de notre fichier et on choisit la méthode<strong> writerow()</strong> pour écrire dessus.
La liste <strong>rating_counter</strong> permet de fixer un seuil pour chaque rating "1000 par exemple" et <strong>full</strong> est un Boolean qui est True quand tous les seuils sont égaux à "1000 par exemple" .
</li>
</ul>

In [ ]:
header = ["Review", "Rating"]

data = open(r'scraped_data.csv', 'w', encoding="utf-8", newline="")

writer = csv.writer(data)

writer.writerow(header)

rating_counter = [0,0,0,0,0]
full = False


<ul>
<li>On définit une fonction<strong>scarp_data()</strong> ayant pour objectif l'extraction des données à partir d'un URL quelconque et insérer ces derniers dans notre fichier CSV. Ce lien URL renvoie vers une page qui contient un certain nombre d'avis qu'on va mettre dans un variable <strong>list_data_reviews</strong>. Pour chaque élémentt de cette liste on cherche l'avis ainsi l'évaluation correspondante. Si pour un rating précis on est déjà arrivé au nombre d'occurence de 1000, on continue vers l'itération suivante. Sinon, on extrait les données et on incrémente la valeur correpondante à ce rating dans la liste <strong>rating_counter</strong>.
</li>
</ul>

In [ ]:
def scrap_data(url):
    page = requests.get(url, headers=headers)
    soup1 = soup(page.content, "html.parser")
    soup2 = soup(soup1.prettify(), 'html.parser')

    list_data_reviews = soup2.findAll('div', {'class': 'cqoFv _T'})

    for data_review in list_data_reviews:
        review = data_review.find('q', {'class': 'XllAv H4 _a'})
        rating = data_review.find('div', {'class': 'emWez F1'})
        rating_value = rating.find('span').attrs['class'][1][7]
        if ( rating_counter[int(rating_value)-1] < 5 ):
            row = [review.find('span').text.strip(), rating_value]
            rating_counter[int(rating_value) - 1] += 1
            writer.writerow(row)

            print(rating_counter)
        else:
            continue

<ul>
<li>On définit une fontcion <strong>number_reviews()</strong> qui détermine le nombre d'avis sur une page </li>
</ul>

In [ ]:
def number_reviews(url):
    page = requests.get(url, headers=headers)
    soup1 = soup(page.content, "html.parser")
    soup2 = soup(soup1.prettify(), 'html.parser')

    list_data_reviews = soup2.findAll('div', {'class': 'cqoFv _T'})
    return len(list_data_reviews)

<ul>
<li>On définit une fonction <strong>last_page()</strong> qui détermine si la page actuelle est la dernière ou pas.</li>
</ul>
 

In [ ]:
def last_page(url):
    page = requests.get(url, headers=headers)
    soup1 = soup(page.content, "html.parser")
    soup2 = soup(soup1.prettify(), 'html.parser')
    pages = soup2.find('div', {'class':'pageNumbers'})
    current_page = pages.find('span', {'class':'pageNum current disabled'})
    current_page_num = int(current_page.text.strip())

    other_pages = pages.findAll('a',{'class':'pageNum'})
    list_other_pages = []
    for other_page in other_pages:
        list_other_pages.append(int(other_page.text.strip()))
    max_page = max(list_other_pages)

    if current_page_num < max_page :
        return False
    else :
        return True


<ul>
<li>Maintenant, on cherche à récuperer tous les hotels se trouvant sur la page principale. On s'intéresse essentiellement aux hotels de la région Parisienne.</li>
</ul>

In [ ]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'}

url = "https://www.tripadvisor.com/Hotels-g187147-Paris_Ile_de_France-Hotels.html"

page = requests.get(url, headers=headers)
soup1 = soup(page.content, "html.parser")
soup2 = soup(soup1.prettify(),'html.parser')

list_hotels = soup2.find('div', {'class':'relWrap'})
hotels = list_hotels.findAll('div', {'class':'prw_rup prw_meta_hsx_responsive_listing ui_section listItem'})

<h3>Enfin !</h3>
<ul>
<li>Pour chaque hotel, on extrait les avis et les évalautions tant que laliste  <strong>rating_counter</strong> n'est pas pleine. On parcourt chaque page à l'aide de la fonction<strong> number_reviews()</strong>  dont on insère le résultat au niveau de URL d'origine et commence on parcourt tt les pages pour chaque hotel.</li>

In [ ]:

for hotel in hotels:

    if ( not full )
    
        link = hotel.find('a', {'class': 'property_title prominent'}, href=True)
        if (link):

            url_origin = "https://www.tripadvisor.com" + link['href']
            url = url_origin
            scrap_data(url)
            cmp = 0
            
            while (not last_page(url) and not full ):

                cmp += number_reviews(url)
                url = re.sub(r'(Reviews-)', "or" + str(cmp) + "-", url_origin)
                scrap_data(url)
                if ( rating_counter[0] == 5 and rating_counter[1] == 5 and rating_counter[2] == 5 and rating_counter[3] == 5 and
                    rating_counter[4] == 5 ):
                    full = True

    else:
        
        break